# TICKER TECHNICAL ANALYSIS

<img src="media/bull_bear.jpeg" alt="Alt Text" width="150"/>

Analsys of buy and sell signals for a given ticker price action for the specified date range. The buy and sell 
signals are based on common technical indicators, custom algorythms & compared against Machine Learning models.

### Imported Modules

In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

### Date Range Ticker


In [2]:
"""
Get current date so we can always get the most recent date from yahoo finance.
date is formated at as YY-mm-dd.

"""


date = datetime.now() + timedelta(days=1)
end_date_fmt = date.strftime("%Y-%m-%d")
start_date = '2022-01-01'
ticker = 'sofi'

### Yahoo API Using Date Range and Ticker

In [3]:
df_daily = yf.download(ticker, start_date, end_date_fmt, progress=False)

In [4]:
"""
The date_range function filters the dataframe for exact date range or span.

"""

def date_range(data, start=None, end=None, span=None):
    if start and end:
        return data[start:end]
    elif span is not None:
        cutoff_date = pd.Timestamp.now()-pd.Timedelta(days=span)
        return data[df_daily.index > cutoff_date]
    else:
        raise ValueError("Please provide either a date range (start='yyyy-mm-dd', end='yyyy-mm-dd') or span")



### Technical Indicators

EMA

In [5]:
""""
The EMA function calculates the Exponential Moving Average (ema) by finding the average between the fast and slow ema.
The Fastema and Slowema are added as columns to the database.

"""

def ema(dataset, fast, slow):
   dataset['Fastema'] = dataset['Close'].ewm(span=fast, adjust=False).mean()
   dataset['Slowema'] = dataset['Close'].ewm(span=slow, adjust=False).mean()
   return dataset

MACD


In [6]:
"""
The macd1 function is calcuating the Moving Average Convergence Divergence (MACD) by finding the difference between 12 and 26 EMA.
Generate singals based on 9 day moving average.
Inserting a macd histogram that shows macd and signal crossing by finding the difference.
"""


def macd1(dataset):
    dataset['macd12'] =  dataset['Close'].ewm(span=12, adjust=False).mean()
    dataset['macd26'] = dataset['Close'].ewm(span=26, adjust=False).mean()
    dataset['macd'] = dataset['macd12'] - dataset['macd26']
    dataset['signal'] = dataset['macd'].ewm(span=9, adjust=False).mean()
    dataset['macd_hist'] = dataset['macd'] - dataset['signal']
    return dataset





RSI

In [7]:
"""
The RSI function calculates the Relative Strength Index (RSI).
The Median_RSI function calculates overbought_level and oversold_level to find the average.

"""

def rsi(dataset, overbought_level, oversold_level):
    delta = dataset['Close'].diff(1)

    # Calculate the exponentially weighted moving averages of gains and losses
    gain_ewm = delta.where(delta > 0, 0).ewm(span=14, adjust=False).mean()
    loss_ewm = -delta.where(delta < 0, 0).ewm(span=14, adjust=False).mean()

    # Calculate the relative strength (RS)
    rs = gain_ewm / loss_ewm

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))
    dataset['RSI'] = rsi

    median_rsi = np.median([overbought_level, oversold_level])
    dataset['Median_RSI'] = np.where(rsi > median_rsi, 1, -1)

    return(dataset)


### Calculating Combined Technical Indicators

In [8]:
# Added a variable called all_indicators that chains/links all technical indicators functions to run all at once.
all_indicators = rsi(macd1(ema(df_daily,5, 7)),70, 30)

In [9]:
# Shows the first 5 rows of all_indicatos dataset.
all_indicators.head()

,Open,High,Low,Close,Adj Close,Volume,Fastema,Slowema,macd12,macd26,macd,signal,macd_hist,RSI,Median_RSI
Date,,,,,,,,,,,,,,,
2022-01-03,16.100,16.129999,15.45,15.68,15.68,27500900,15.680000,15.680000,15.680000,15.680000,0.000000,0.000000,0.000000,NaN,-1
2022-01-04,15.610,15.861000,14.59,15.05,15.05,34175900,15.470000,15.522500,15.583077,15.633334,-0.050256,-0.010051,-0.040205,0.0,-1
2022-01-05,14.880,15.230000,14.04,14.10,14.10,35103100,15.013334,15.166875,15.354912,15.519753,-0.164842,-0.041009,-0.123832,0.0,-1
2022-01-06,14.016,14.270000,13.01,13.78,13.78,46269400,14.602222,14.820156,15.112617,15.390883,-0.278265,-0.088461,-0.189805,0.0,-1
2022-01-07,13.821,14.400000,13.62,13.74,13.74,28716200,14.314815,14.550117,14.901445,15.268595,-0.367150,-0.144198,-0.222951,0.0,-1
